In [1]:
# Importing the necessary libraries

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import random

In [2]:
# Creating a function to load in each maze
def load_image(num):
    image = cv2.imread("[local path to maze files]" + num + ".png") # Loading the image
    image = image[:,:,0]+image[:,:,1]+image[:,:,2] # Summing the layers

    # Creating the maze array to read the image
    maze_array  = np.zeros((21,21))

    # Reducing an image array down to a maze state array
    for i in range(len(maze_array)):
        for j in range(len(maze_array)):
            if image[0+i*20+1,0+j*20+1] == 0.0:
                maze_array[i,j] = 0.0
            else:
                maze_array[i,j] = 253.0

    # Differentiating the starting and ending points
    maze_array[1,0] = 30.0
    maze_array[19,20] = 120.0

    return maze_array,1,0,reward_matrix()

# Creating the reward distribution
def reward_matrix():

    # Setting the reward scaling factor
    scale = 40

    # filling the array using radial distance (rounded)
    rewards = np.zeros((21,21))
    for i in range(len(rewards)):
        for j in range(len(rewards)):
            rewards[i,j] = scale-round(np.sqrt((19-i)**2+(19-j)**2))
    
    return rewards

# Given a maze state and position, scanning for possible actions
def compute_actions(state,row,col):
    actions = []
    if state[row+1,col] != 0 and state[row+1,col] != 30 and state[row+1,col] != 120 and state[row+1,col] != 200:
        actions.append("down")
    if state[row-1,col] != 0 and state[row-1,col] != 30 and state[row-1,col] != 120 and state[row-1,col] != 200:
        actions.append("up")
    if state[row,col+1] != 0 and state[row,col+1] != 30 and state[row,col+1] != 120 and state[row,col+1] != 200:
        actions.append("right")
    if state[row,col-1] != 0 and state[row,col-1] != 30 and state[row,col-1] != 120 and state[row,col-1] != 200:
        actions.append("left")
    return actions

# After selecting a state, proceeding in the direction and updating the maze state
def take_step(state,action,row,col):
    state[row,col] = 30
    if action == "right":
        state[row,col+1] = 200
        return state,row,col+1
    if action == "left":
        state[row,col-1] = 200
        return state,row,col-1
    if action == "up":
        state[row-1,col] = 200
        return state,row-1,col
    if action == "down":
        state[row+1,col] = 200
        return state,row+1,col

# Creating a function to repeatedly wander a branch using Monte Carlo simulation
def random_wander(state,row,col,rewards):

    # Setting an intial reward value
    reward = 0

    # Testing the random path seeking (as many as 15 steps ahead)
    for i in range(15):
        # Listing possible actions
        actions = compute_actions(state,row,col)

        # Checking if the route meets the exit
        if row == 19 and col == 19:
            # Providing a decisive reward
            reward += 10000
            # Ending the path
            break
            
        # Testing if you've hit a dead end
        if len(actions) == 0:
            state[row,col] = 200
            # Ending the path
            break
        else:
            # Choosing a random traversal direction
            state, row, col = take_step(state,random.choice(actions),row,col)
        reward += rewards[row,col]

    return round(reward)

# Creating a method to evaluate each possible action at an intersection
def test_actions(state, actions, row, col,rewards_array):
    # Creating a list to store the average reward down each path
    rewards = []

    # Duplicating the state for each possible action
    for path in actions:
        temp_rewards = []
        temp_copy = np.copy(state)
        # Taking the action
        temp_copy, temp_row, temp_col = take_step(temp_copy, path, row, col)

        # Proceeding down 100 random paths along a gievn branch
        for i in range(100):  
            reward = random_wander(np.copy(temp_copy), temp_row, temp_col,rewards_array)
            temp_rewards.append(reward)
        
        # rewards.append(np.max(temp_rewards)) # Returning the maximum reward
        rewards.append(np.mean(temp_rewards)) # Returning the average reward

    # Returning the index of the "best" action
    return rewards.index(max(rewards))

# Creating a method to solve a given puzzle
def solve_puzzle(state,row,col,rewards):
    # Allows for up to 1500 episodes (no path length restrictions)
    for i in range(1500):

        # If the agent has reached the end, end the loop
        if row == 19 and col == 19:
            return 1
            break
    
        # Finding the possible actions in the current state
        actions = compute_actions(state,row,col)
        # Showing failure if stuck in dead end
        if len(actions) == 0:
            return 0
            break
        # Following a path if not at intersection
        if len(actions) == 1:
            state, row, col = take_step(state,actions[0],row,col)
        # If at intersection, using Monte Carlo simulation to decide path
        else:
            act = test_actions(state,actions,row,col,rewards)
            state, row, col = take_step(state,actions[act],row,col)

In [3]:
# Using a counter to track successful solutions
correct = 0

# Iterating through solving each puzzle
for puzzle in range(1000):
    state, row, col, rewards = load_image(str(puzzle))
    correct += solve_puzzle(state,row,col,rewards)

# Displaying the algorithm's accuracy
print(correct/1000)

0.427
